In [ ]:
import comodoro
escogido, m, n = 3, 0, 0
print("¡Hola! Bienvenido al simulador de Comodoro. Por favor, ingresa el número del mapa con el que quieres trabajar")
escogido = int(input("Ingrese un número del 1 al 4: "))
iteraciones = int(input("¡Perfecto! ahora, ingresa el número de iteraciones para la simulación: "))
mapa = comodoro.generar_mapa(escogido)
for i in mapa:
    m += 1
for j in mapa[1]:
    n += 1

In [ ]:
#crear matriz auxiliar que tiene 5 a los bordes, para reconocerlos al momento de la simulacion
aux = []
nueva = []
#agregar fila de arriba
for i in range(0,n+2):
    nueva.append(5)
aux.append(nueva)
nueva = []
#agregar filas del medio
for i in range(0,m):
    nueva.append(5)
    for j in range(0,n):
        nueva.append(mapa[i][j])
    nueva.append(5)
    aux.append(nueva)
    nueva = []
#agregar fila de abajo
for i in range(0,n+2):
    nueva.append(5)
aux.append(nueva)
nueva = []

In [ ]:
#crear matriz de estado, inicialmente llena de ceros y de dimensiones (m+2)x(n+2)
estado = []
for i in range(0,m+2):
    for j in range(0,n+2):
        nueva.append(0)
    estado.append(nueva)
    nueva = []
nueva = []

In [ ]:
#clase vehiculo, almacena fila y columna
class vehiculo:
    def __init__(self, f, c):
        self.f = f #fila
        self.c = c #columna
    
    #funcion que entrega la casilla final de un turno
    def sgte_casilla(self,vel, aux, vial):
        #up
        mov = vel
        while mov > 0:
            if self.f > 0:
                if aux[f-1][c] == vial:
                    a = True
                if vial == 5:
                    if aux[f-1][c] == 2 or aux[f-1][c] == 3 or aux[f-1][c] == 4:
                        a = True
                        newvial = aux[f-1][c]
                        vial = newvial
            else:
                a = False
            #down
            if self.f < m+1:
                if aux[f+1][c] == vial:
                    b = True
                if vial == 5:
                    if aux[f+1][c] == 2 or aux[f+1][c] == 3 or aux[f+1][c] == 4:
                        b = True
                        newvial = aux[f+1][c]
                        vial = newvial
            else:
                b = False
            #left
            if self.c > 0:
                if aux[f][c-1] == vial:
                    i = True
                if vial == 5:
                    if aux[f][c-1] == 2 or aux[f][c-1] == 3 or aux[f][c-1] == 4:
                        i = True
                        newvial = aux[f][c-1]
                        vial = newvial
            else:
                i = False
            #right
            if self.c < n+1:
                if aux[f][c+1] == vial:
                    d = True
                if vial == 5:
                    if aux[f][c+1] == 2 or aux[f][c+1] == 3 or aux[f][c+1] == 4:
                        d = True
                        newvial = aux[f][c+1]
                        vial = newvial
            else:
                d = False
            choice = comodoro.ruta(a,b,i,d)
            if choice == a:
                self.f -= 1
            elif choice == b:
                self.f += 1
            elif choice == i:
                self.c -= 1
            elif choice == d:
                self.d += 1
            if aux[self.f][self.c] == 5:
                mov = 0
            mov -= 1
            if mov == 0:
                print("Se ha movido un vehiculo")
            return self.f, self,c

            #finalmente esta funcion me va a entregar la fila y la columna final del vehiculo por turno
        

In [ ]:
class caminata(vehiculo):
    def __init__(self, f, c, vial):
        super().__init__(f,c)
        self.ocup = 1
        if vial == "camino" or vial == "calle" or vial == "autopista":
            self.vel = 1
        else:
            self.vel = 0

In [ ]:
class bicicleta(vehiculo):
    def __init__(self,f,c,vial):
        super().__init__(f,c)
        self.ocup = round(comodoro.uniforme(1,2))
        if vial == "camino":
            self.vel = 2
        elif vial == "calle" or vial == "autopista":
            self.vel = 3        
        else:
            self.vel = 0

In [ ]:
class auto(vehiculo):
    def __init__(self,f,c, vial):
        super().__init__(f,c)
        self.ocup = round(comodoro.uniforme(1,5))
        if vial == "camino":
            self.vel = 3
        elif vial == "calle":
            self.vel = 5
        elif vial == "autopista":
            self.vel = 7
        else:
            self.vel = 0

In [ ]:
class bus(vehiculo):
    def __init__(self,f,c, vial):
        super().__init__(f,c)
        self.ocup = round(comodoro.uniforme(1,40))
        if vial == "camino":
            self.vel = 2
        elif vial == "calle":
            self.vel = 4
        elif vial == "autopista":
            self.vel = 5
        else:
            self.vel = 0

In [ ]:
class celda:
    def __init__(self, f ,c, matriz):
        self.f = f
        self.c = c
        self.matriz = matriz
        self.value = matriz[f][c]

In [ ]:
class edificio(celda):
    def __init__(self, f, c, matriz):
        super().__init__(f, c, matriz)
        self.turns = 0
        if self.value == -1:
            self.tipo = "vacia"
        elif self.value == 0:
            self.tipo = "hogar"
        elif self.value == 1:
            self.tipo = "oficina"
        elif self.value == 5:
            self.tipo = "borde"
        
    def actualizar_turnos(self):
        if self.turns == 0:
            if self.tipo == "hogar":
                self.turns = int(comodoro.exponencial(1))
            if self.tipo == "oficina":
                self.turns = int(comodoro.exponencial(2))
            if self.tipo == "borde":
                self.turns = int(comodoro.exponencial(3))   
        else:
            self.turns -= 1
    
    def boolgen(self):
        if self.turns == 0:
            print("Se ha generado un viaje")
            return True
        else:
            return False

    def dir_viaje(self):
        if self.f > 0:
            if self.matriz[self.f-1][self.c] != -1 or self.matriz[self.f-1][self.c] != 0 or self.matriz[self.f-1][self.c] != 1:
                direc = "up"
        if self.f < m+1:
            if self.matriz[self.f+1][self.c] != -1 or self.matriz[self.f+1][self.c] != 0 or self.matriz[self.f+1][self.c] != 1:
                direc = "down"
        if self.c > 0:
            if self.matriz[self.f][self.c-1] != -1 or self.matriz[self.f][self.c-1] != 0 or self.matriz[self.f][self.c-1] != 1:
                direc = "left"
        if self.c < n+1:
            if self.matriz[self.f][self.c+1] != -1 or self.matriz[self.f][self.c+1] != 0 or self.matriz[self.f][self.c+1] != 1:
                direc = "right"
        else:
            direc = "none"
        return direc

#a = edificio(6,20,aux)
#print(a.value)
#print(aux[6][20])
#print(a.tipo)
#print(a.dir_viaje())

In [ ]:
class vialidad(celda):
    def __init__(self,f,c,aux):
        super().__init__(f, c, aux)
        #que esto sea como un contador automatico...
        self.vehiculos = []
        if self.value == 2:
            self.tipo = "camino"
        elif self.value == 3:
            self.tipo = "calle"
        elif self.value == 4:
            self.tipo = "autopista"
        #considero a borde como edificio y vialidad, porque puede generar viajes y almacenar gente a la vez
        elif self.value == 5:
            self.tipo = "borde"
        else:
            self.tipo = "error"
        
    def ocup_actual(self):
        gente = 0
        for i in range(0,len(self.vehiculos)):
            gente += self.vehiculos[i].ocup
        return gente
        
    
#b = vialidad(6,21,aux)
#print(b.value)
#print(aux[6][21])
#print(b.tipo)
#bus1 = bus(6,21,"calle")
#b.vehiculos.append(bus1)
#print(bus1.ocup)
#print(b.vehiculos[0].ocup)
#print(b.ocup_actual())

In [ ]:
#estas funciones va a llevarme la simulacion, en primer lugar creo listas con todos los edificios y las vialidades
#edificios tieien su fila, columna, valor, tipo
#vialidad tienen su fila, columna, tipo, gente, vehiculos
t = 0 #numero de turnos
#crear listas
edificios = []
vias = []
for i in range(0,m+2):
    for j in range(0,n+2):
        if aux[i][j] == 0 or aux[i][j] == 1 or aux[i][j] == 5:
            edificios.append(edificio(i,j,aux))
        elif aux[i][j] == 3 or aux[i][j] == 4:
            vias.append(vialidad(i,j,aux))

#print(edificios[100].tipo)
#print(vias[0].tipo)

In [ ]:
def gen_viaje(edificios, vias, aux):
    for k in range(0, len(edificios)):
        if edificios[k].boolgen() == True:
            direc = edificios[k].dir_viaje() #esto me entrega up, down, left, right
            fil = edificios[k].f
            col = edificios[k].c
            a = round(comodoro.uniforme(1,100))
            if a in range(1,15):
                modo = "caminata"
            elif a in range(16, 25):
                modo = "bicicleta"
            elif a in range(26, 50):
                modo = "auto"
            elif a in range(51,100):
                modo = "bus"
            else:
                modo = "none"
            if direc == "up":
                for via in range(0, len(vias)):
                    if vias[via].f == fil-1 and vias[via].c == col:
                        if modo == "caminata":
                            vias[via].vehiculos.append(caminata(fil-1,col,"caminata"))
                        elif modo == "bicicleta":
                            vias[via].vehiculos.append(bicicleta(fil-1,col,"bicicleta"))
                        elif modo == "bus":
                            vias[via].vehiculos.append(bus(fil-1,col,"bus"))
                        elif modo == "auto":
                            vias[via].vehiculos.append(auto(fil-1,col,"auto"))
            if direc == "down":
                for via in range(0, len(vias)):
                    if vias[via].f == fil+1 and vias[via].c == col:
                        if modo == "caminata":
                            vias[via].vehiculos.append(caminata(fil+1,col,"caminata"))
                        elif modo == "bicicleta":
                            vias[via].vehiculos.append(bicicleta(fil+1,col,"bicicleta"))
                        elif modo == "bus":
                            vias[via].vehiculos.append(bus(fil+1,col,"bicicleta"))
                        elif modo == "auto":
                            vias[via].vehiculos.append(auto(fil+1,col,"auto"))
            if direc == "left":
                for via in range(0, len(vias)):
                    if vias[via].f == fil and vias[via].c == col-1:
                        if modo == "caminata":
                            vias[via].vehiculos.append(caminata(fil,col-1,"caminata"))
                        elif modo == "bicicleta":
                            vias[via].vehiculos.append(bicicleta(fil,col-1,"bicicleta"))
                        elif modo == "bus":
                            vias[via].vehiculos.append(bus(fil,col-1,"bus"))
                        elif modo == "auto":
                            vias[via].vehiculos.append(auto(fil,col-1,"auto"))
            if direc == "right":
                for via in range(0,len(vias)):
                    if vias[via].f == fil and vias[via].c == col+1:
                        if modo == "caminata":
                            vias[via].vehiculos.append(caminata(fil,col+1,"caminata"))
                        if modo == "bicicleta":
                            vias[via].vehiculos.append(bicicleta(fil,col+1,"bicicleta"))
                        if modo == "bus":
                            vias[via].vehiculos.append(bus(fil,col+1,"bus"))
                        if modo == "auto":
                            vias[via].vehiculos.append(auto(fil,col+1,"auto"))

                            
def actualizar_vias(vias):
    for via in range(0,len(vias)):
        for vehic in range(0,len(vias[via].vehiculos)-5):
            veh = vias[via].vehiculos[vehic]
            fil = veh.f
            col = veh.c
            vel = veh.vel
            vial = aux[fil][col]
            veh.sgte_casilla(vel,aux,vial)
            newfil = veh.f
            newcol = veh.c
            for newvia in range (0,len(vias)):
                if vias[newvia].f == newfil and vias[newvia].c == newcol:
                    lastvia = vias[newvia]
            lastvia.vehiculos.append(veh)
            vias[via].vehiculos.remove(veh)
            #vias[via].vehiculos.append(veh)

            
#            for newvia in range(0,len(vias)):
 #               if vias[via].f == newfil and vias[via].c== newcol:
  #                  vias[newvia].vehiculos.append(veh)
            #vias[via].vehiculos.remove(veh)
                    
#def actualizar_vias(vias):
#recorro la lista de calles
 #   for via in range(0,5):
  #      #recorro los vehiculos para una calle en particular
   #     for veh in range(0,5):
    #        # print(vias[via].vehiculos[veh])
     #       veh = vias[via].vehiculos[veh]
      #      fil = veh.f
       #     col = veh.c
        #    vel = veh.vel
         #   vial = aux[fil][col]
          #  veh.sgte_casilla(vel,aux,vial)
           # newfil = veh.f
#            newcol = veh.c
 #           vias[via].vehiculos.remove(veh)
  #          for newvia in range(0,len(vias)):
   #             if vias[via].f == newfil and vias[via].c == newcol:
    #                vias[newvia].vehiculos.append(veh)
                            
def actualizar_estado(vias, estado):
    for k in range(0, len(vias)):
        fil = vias[k].f
        col = vias[k].c
        estado[fil][col] = vias[k].ocup_actual()

In [ ]:
for i in range(0,iteraciones):
    gen_viaje(edificios,vias,aux)
    actualizar_vias(vias)
    actualizar_estado(vias,estado)
    comodoro.imprimir_mapa(aux,estado)
    